In [1]:
from hashlib import sha256
import time
from math import log2
from typing import List

MIN_DIFFICULTY = 12
MIN_STOCH_DIFFICULTY = 21
PERIOD = 60
BASE_TRANSACTION_FEE = 1

In [2]:
def count_leading_zeros(sha256_digest):
    binary_representation = ''.join(f'{byte:08b}' for byte in sha256_digest)
    leading_zeros = len(binary_representation) - len(binary_representation.lstrip('0'))
    return leading_zeros

In [3]:
class Challenge:
    def __init__(self, challenge):
        self.challenge = challenge
        self.last_solved = time.time()
        self.reward = 0

    def submit(self, solution):
        digest = sha256(self.challenge.encode() + solution.encode())
        leading_zeros = count_leading_zeros(digest.digest())
        self.challenge = digest.hexdigest()
        reward = 0
        curr_time = time.time()
        liveness_penalty = 0
        # print(f"Leading zeros: {leading_zeros}")
        if leading_zeros >= MIN_DIFFICULTY:
            reward = 2**leading_zeros
            liveness_penalty = (curr_time - (self.last_solved + PERIOD)) / PERIOD
            reward  = max(0, reward - reward * liveness_penalty)
        if curr_time - PERIOD < self.last_solved:
            print("Too soon to submit another solution")
            reward = 0
        self.reward += reward - BASE_TRANSACTION_FEE
        self.last_solved = curr_time
        return reward, liveness_penalty

In [4]:
class NormalSolver:
    def __init__(self, challenge: Challenge):
        self.challenge = challenge
        self.terminatation_time = time.time() + PERIOD * 120 #2hrs

    def submit(self, best):
        reward, liveness_penalty = self.challenge.submit(str(best[1]))
        print( "Liveness Penalty: ", liveness_penalty, "Submitteed Solution with reward: ", reward, " and total reward: ", self.challenge.reward, "Difficulty: ", best[0])


    def solve(self):
        sol = 0
        target_time = self.challenge.last_solved + PERIOD
        best = (0, 0)
        while time.time() < target_time or best[0] < MIN_DIFFICULTY:
            digest = sha256(self.challenge.challenge.encode() + str(sol).encode()).digest()
            leading_zeros = count_leading_zeros(digest)
            if leading_zeros > best[0]:
                best = (leading_zeros, sol)
            sol += 1
        self.submit(best)

        if time.time() < self.terminatation_time:
            self.solve()

In [5]:
class StochasticSolver:
    def __init__(self, challenges: List[Challenge], multiple: int = 2):
        self.challenges = challenges
        self.multiplier = multiple
        self.sols = [0 for _ in challenges]
        self.bests = [(0, 0) for _ in challenges]
        self.terminatation_time = time.time() + PERIOD * 120 #2hrs

    def submit(self):
        liveness_penalty = 0
        rewards = 0
        total_rewards = 0
        for best, challenge in zip(self.bests, self.challenges):
            reward, liveness_penalty = challenge.submit(str(best[1]))
            rewards += reward
            total_rewards += challenge.reward
        print("Total liveness penalty: ", liveness_penalty, "Rewards: ", rewards, "Total rewards: ", total_rewards)

    def solve(self):
        curr = 0
        curr_count = 0
        endtimes = []
        for ind in range(len(self.challenges)):
            endtimes.append(time.time() + (PERIOD/len(self.challenges)) * (ind + 1))

        while time.time() < self.challenges[-1].last_solved + PERIOD:
            if time.time() > endtimes[curr] and curr < len(self.challenges) - 1:
                curr += 1
                curr_count = 0
            sol = self.sols[curr]
            digest = sha256(self.challenges[curr].challenge.encode() + str(sol).encode()).digest()
            leading_zeros = count_leading_zeros(digest)
            curr_count += 1
            self.sols[curr] += 1
            if leading_zeros > self.bests[curr][0]:
                self.bests[curr] = (leading_zeros, sol)

                # ***********************************************************************#
                expected_difficulty = log2(curr_count) # This is the key to the algorithm
                # ***********************************************************************#

                if leading_zeros >= expected_difficulty + log2(self.multiplier) and curr < len(self.challenges) - 1 and leading_zeros >= MIN_STOCH_DIFFICULTY:
                    print("leading zeros: ", leading_zeros, "expected difficulty: ", expected_difficulty, "curr_count: ", curr_count, "curr: ", curr, "challenge: ", self.challenges[curr].challenge)
                    curr_count = 0
                    curr += 1

        self.submit()
        if time.time() < self.terminatation_time:
            self.sols = [0 for _ in self.challenges]
            self.bests = [(0, 0) for _ in self.challenges]
            self.solve()








In [6]:
# Simulate a normal miner submiting best hash every PERIOD seconds
normal_solver = NormalSolver(Challenge('f20jfjjv8w100'))
normal_solver.solve()

Liveness Penalty:  5.841255187988282e-07 Submitteed Solution with reward:  2097150.775  and total reward:  2097149.775 Difficulty:  21
Liveness Penalty:  7.112820943196615e-07 Submitteed Solution with reward:  67108816.266666666  and total reward:  69205965.04166667 Difficulty:  26
Liveness Penalty:  7.112820943196615e-07 Submitteed Solution with reward:  8388602.033333333  and total reward:  77594566.075 Difficulty:  23
Liveness Penalty:  7.62939453125e-07 Submitteed Solution with reward:  1048575.2  and total reward:  78643140.275 Difficulty:  20
Liveness Penalty:  7.708867390950521e-07 Submitteed Solution with reward:  1048575.1916666667  and total reward:  79691714.46666667 Difficulty:  20
Liveness Penalty:  6.556510925292969e-07 Submitteed Solution with reward:  2097150.625  and total reward:  81788864.09166667 Difficulty:  21
Liveness Penalty:  6.119410196940105e-07 Submitteed Solution with reward:  268435291.73333332  and total reward:  350224154.825 Difficulty:  28
Liveness Pen

In [7]:
# Simulate a stochastic miner submiting best hashes every PERIOD seconds
challenges = [
    Challenge('0agtadvbr35'),
    Challenge('04afwee245'),
    Challenge('0743aqwrty7@5'),
    Challenge('qefafqrwerwrerf3'),
    Challenge('04afwee242313fqfqef5'),
    Challenge('0743aq12fewfafefwrty7@5'),
]

stochastic_solver = StochasticSolver(challenges)

stochastic_solver.solve()

Total liveness penalty:  2.6702880859375e-06 Rewards:  3801079.4244791665 Total rewards:  3801073.4244791665
leading zeros:  22 expected difficulty:  18.662178095194136 curr_count:  414835 curr:  0 challenge:  0000481aba3b6c574ef387b191998bc7ca490a1ac53bc6cfc9c3982b5b50976e
leading zeros:  21 expected difficulty:  18.2973954606884 curr_count:  322155 curr:  4 challenge:  000039de87368239ae94b38e71397352373aa3d197677ca922ea45eab4e47e00
Total liveness penalty:  2.280871073404948e-06 Rewards:  12058595.744791666 Total rewards:  15859663.169270832
leading zeros:  21 expected difficulty:  18.441935064214988 curr_count:  356103 curr:  1 challenge:  000008525e70b9b43c62030667e0720d5ad1e759278659ab3900573cbdf89c1f
Total liveness penalty:  4.116694132486979e-06 Rewards:  3932147.3765625004 Total rewards:  19791804.545833334
leading zeros:  22 expected difficulty:  17.237303338470465 curr_count:  154506 curr:  0 challenge:  00000bb94a939fd77664758a904ae74ee60135f6da9eeef2d3312c5925fd5d41
leading